# Project-2
## Tanvir Khan, Nicky Pant, Paul Pineda, James Ye, Fabienne Zumbuehl
### loadCSVFiles

In [1]:
import pandas as pd
#from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from sqlalchemy import create_engine
import psycopg2

In [2]:
# Creating connection with database
# StocksDataBase is database name for this project
connection_string = "postgres:postgres@localhost:5432/StocksDataBase"
engine = create_engine(f'postgresql://{connection_string}')

In [17]:
# Confirm tables
engine.table_names()

['company', 'price']

## load price csv files

In [4]:
df_600104 = pd.read_csv("data/600104.SS.csv");
df_600104.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-06-30,20.799999,22.820000,20.700001,22.600000,16.272430,123950995
1,2015-07-01,22.100000,22.580000,21.450001,21.620001,15.566809,91470020
2,2015-07-02,21.790001,22.110001,20.600000,21.150000,15.228400,90836062
3,2015-07-03,21.250000,21.510000,19.580000,20.260000,14.587584,98059183
4,2015-07-06,22.250000,22.260000,21.320000,21.900000,15.768413,226156959


In [5]:
print(df_600104.dtypes)

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object


In [6]:
ticker = "600104.SS"
df_600104["ticker"] = ticker
df_600104.head()

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2015-06-30,20.799999,22.820000,20.700001,22.600000,16.272430,123950995,600104.SS
1,2015-07-01,22.100000,22.580000,21.450001,21.620001,15.566809,91470020,600104.SS
2,2015-07-02,21.790001,22.110001,20.600000,21.150000,15.228400,90836062,600104.SS
3,2015-07-03,21.250000,21.510000,19.580000,20.260000,14.587584,98059183,600104.SS
4,2015-07-06,22.250000,22.260000,21.320000,21.900000,15.768413,226156959,600104.SS


In [7]:
# rename column heading to match database table columns
df_600104.rename(columns = {"Date":"date", "Open":"open", "High":"high", "Low":"low", "Close":"close", "Adj Close":"adj_close", "Volume":"volume"}, inplace=True)
df_600104.head()

,date,open,high,low,close,adj_close,volume,ticker
0,2015-06-30,20.799999,22.820000,20.700001,22.600000,16.272430,123950995,600104.SS
1,2015-07-01,22.100000,22.580000,21.450001,21.620001,15.566809,91470020,600104.SS
2,2015-07-02,21.790001,22.110001,20.600000,21.150000,15.228400,90836062,600104.SS
3,2015-07-03,21.250000,21.510000,19.580000,20.260000,14.587584,98059183,600104.SS
4,2015-07-06,22.250000,22.260000,21.320000,21.900000,15.768413,226156959,600104.SS


In [8]:
# make ticker as the first column
df_600104 = df_600104[["ticker", "date", "open", "high", "low", "close", "adj_close", "volume"]]
df_600104.head()

,ticker,date,open,high,low,close,adj_close,volume
0,600104.SS,2015-06-30,20.799999,22.820000,20.700001,22.600000,16.272430,123950995
1,600104.SS,2015-07-01,22.100000,22.580000,21.450001,21.620001,15.566809,91470020
2,600104.SS,2015-07-02,21.790001,22.110001,20.600000,21.150000,15.228400,90836062
3,600104.SS,2015-07-03,21.250000,21.510000,19.580000,20.260000,14.587584,98059183
4,600104.SS,2015-07-06,22.250000,22.260000,21.320000,21.900000,15.768413,226156959


In [9]:
# insert ticker into company table if it doesn't exist yet, because it is the foreign key for the price table
connection = engine.connect()
result = connection.execute(f"select * from company where ticker = '{ticker}'")
rows = result.fetchall()
result.close()
if len(rows) == 0:
    print(f"inserting {ticker} into company table")
    connection.execute(f"insert into company(ticker) values ('{ticker}')")
    result = connection.execute(f"select * from company where ticker = '{ticker}'")
    rows = result.fetchall()
    result.close()
for row in rows:
    print(row)
    


('600104.SS', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)


In [16]:
# load dataframe into database
try:
    df_600104.to_sql(name='price', con=engine, if_exists='append', chunksize = 5, index=False)
except Exception as e:
    print(e)
price_df = pd.read_sql("select * from price", con=engine)
price_df.head()
price_df.count()

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "price_pkey"
DETAIL:  Key (ticker, date)=(600104.SS, 2015-06-30) already exists.

[SQL: INSERT INTO price (ticker, date, open, high, low, close, adj_close, volume) VALUES (%(ticker)s, %(date)s, %(open)s, %(high)s, %(low)s, %(close)s, %(adj_close)s, %(volume)s)]
[parameters: ({'ticker': '600104.SS', 'date': '2015-06-30', 'open': 20.799999, 'high': 22.82, 'low': 20.700001, 'close': 22.6, 'adj_close': 16.27243, 'volume': 123950995}, {'ticker': '600104.SS', 'date': '2015-07-01', 'open': 22.1, 'high': 22.58, 'low': 21.450001, 'close': 21.620001000000002, 'adj_close': 15.566809, 'volume': 91470020}, {'ticker': '600104.SS', 'date': '2015-07-02', 'open': 21.790001, 'high': 22.110001, 'low': 20.6, 'close': 21.15, 'adj_close': 15.2284, 'volume': 90836062}, {'ticker': '600104.SS', 'date': '2015-07-03', 'open': 21.25, 'high': 21.51, 'low': 19.58, 'close': 20.26, 'adj_close': 14.587584, 'volume': 98059183}, {'ticker': 

ticker       1215
date         1215
open         1215
high         1215
low          1215
close        1215
adj_close    1215
volume       1215
dtype: int64

In [22]:
import glob
priceCSVFiles = glob.glob("data/*.csv")
priceCSVFiles

['data\\600104.SS.csv',
 'data\\ACGBY.csv',
 'data\\ALIZF.csv',
 'data\\AXAHF.csv',
 'data\\BAC.csv',
 'data\\BACHY.csv',
 'data\\BMWYY.csv',
 'data\\CAH.csv',
 'data\\CHL.csv',
 'data\\CICHF.csv',
 'data\\COST.csv',
 'data\\CRWOY.csv',
 'data\\CWYCF.csv',
 'data\\DMLRY.csv',
 'data\\FNMA.csv',
 'data\\GE (1).csv',
 'data\\GE.csv',
 'data\\GM.csv',
 'data\\GOOG.csv',
 'data\\HMC.csv',
 'data\\JPHLF.csv',
 'data\\JPM.csv',
 'data\\KR.csv',
 'data\\LFC.csv',
 'data\\LUKOY.csv',
 'data\\MSFT.csv',
 'data\\MUFG.csv',
 'data\\OGZPY.csv',
 'data\\PSX.csv',
 'data\\VLO.csv',
 'data\\VZ.csv',
 'data\\WBA.csv']

In [45]:
# loop through all data/*.csv files and load them into price table
for file in priceCSVFiles:
    file = file.strip()
    
    print(file)
    price_df = pd.read_csv(file)
    print(price_df.head())
    csv_file = file.split('\\')[1]
    
    # ignore company.csv, we will handle it differetly
    if csv_file == "company.csv":
        continue

    ticker = csv_file.strip('.csv')
    price_df["ticker"] = ticker
    print(ticker)
    
    # rename column heading to match database table columns
    price_df.rename(columns = {"Date":"date", "Open":"open", "High":"high", "Low":"low", "Close":"close", "Adj Close":"adj_close", "Volume":"volume"}, inplace=True)
    # make ticker as the first column
    price_df = price_df[["ticker", "date", "open", "high", "low", "close", "adj_close", "volume"]]
    
    # insert ticker into company table if it doesn't exist yet, because it is the foreign key for the price table
    connection = engine.connect()
    result = connection.execute(f"select * from company where ticker = '{ticker}'")
    rows = result.fetchall()
    result.close()
    if len(rows) == 0:
        print(f"inserting {ticker} into company table")
        connection.execute(f"insert into company(ticker) values ('{ticker}')")
        result = connection.execute(f"select * from company where ticker = '{ticker}'")
        rows = result.fetchall()
        result.close()
    for row in rows:
        print(row)
    
    # load dataframe into database
    try:
        price_df.to_sql(name='price', con=engine, if_exists='append', chunksize = 20, index=False)
    except Exception as e:
        print(e)

# check out the price table
price_df2 = pd.read_sql("select * from price", con=engine)
price_df2.head()
price_df2.count()

data\600104.SS.csv
         Date       Open       High        Low      Close  Adj Close  \
0  2015-06-30  20.799999  22.820000  20.700001  22.600000  16.272430   
1  2015-07-01  22.100000  22.580000  21.450001  21.620001  15.566809   
2  2015-07-02  21.790001  22.110001  20.600000  21.150000  15.228400   
3  2015-07-03  21.250000  21.510000  19.580000  20.260000  14.587584   
4  2015-07-06  22.250000  22.260000  21.320000  21.900000  15.768413   

      Volume  
0  123950995  
1   91470020  
2   90836062  
3   98059183  
4  226156959  
600104.SS
('600104.SS', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "price_pkey"
DETAIL:  Key (ticker, date)=(600104.SS, 2015-06-30) already exists.

[SQL: INSERT INTO price (ticker, date, open, high, low, close, adj_close, volume) VALUES (%(ticker)s, %(date)s, %(open)s, %(high)s, %(low)s, %(close)s, %(adj_close)s, %(volume)s)]
[

data\CRWOY.csv
         Date       Open       High        Low      Close  Adj Close  Volume
0  2015-06-29  27.030001  27.030001  27.030001  27.030001  24.905254       0
1  2015-06-30  27.030001  27.030001  27.030001  27.030001  24.905254       0
2  2015-07-01  27.350000  27.350000  27.350000  27.350000  25.200098     400
3  2015-07-02  27.350000  27.350000  27.350000  27.350000  25.200098       0
4  2015-07-06  27.350000  27.350000  27.350000  27.350000  25.200098       0
CRWOY
inserting CRWOY into company table
('CRWOY', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
data\CWYCF.csv
         Date  Open  High   Low  Close  Adj Close  Volume
0  2015-06-29  1.49  1.53  1.49   1.53   1.395204    2000
1  2015-06-30  1.52  1.52  1.52   1.52   1.386085   23700
2  2015-07-01  1.52  1.52  1.52   1.52   1.386085       0
3  2015-07-02  1.52  1.52  1.52   1.52   1.386085       0
4  2015-07-06  1.52  1.52  1.52   1.52   1.386085    2000
CWYCF
inserting CWY

data\MSFT.csv
         Date       Open       High        Low      Close  Adj Close    Volume
0  2015-06-29  45.040001  45.230000  44.360001  44.369999  40.155910  34081700
1  2015-06-30  44.709999  44.720001  43.939999  44.150002  39.956821  35945400
2  2015-07-01  44.459999  45.230000  44.099998  44.450001  40.228317  28343900
3  2015-07-02  44.480000  44.750000  44.060001  44.400002  40.183064  21752000
4  2015-07-06  43.959999  44.480000  43.950001  44.389999  40.174019  23034000
MSFT
inserting MSFT into company table
('MSFT', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
data\MUFG.csv
         Date  Open  High   Low  Close  Adj Close   Volume
0  2015-06-29  7.29  7.30  7.21   7.22   6.262765  1273500
1  2015-06-30  7.25  7.26  7.18   7.22   6.262765  1021500
2  2015-07-01  7.31  7.34  7.29   7.30   6.332158   992100
3  2015-07-02  7.27  7.29  7.23   7.29   6.323483   838900
4  2015-07-06  7.22  7.27  7.20   7.23   6.271439   891300
MUFG
i

ticker       40128
date         40128
open         40128
high         40128
low          40128
close        40128
adj_close    40128
volume       40128
dtype: int64